In [1]:
import copy
import math
import random
import numpy as np

from abc import ABC
from typing import List

from jmetal.algorithm.multiobjective.nsgaii import NSGAII
from jmetal.core.operator import Crossover, Mutation
from jmetal.core.problem import IntegerProblem
from jmetal.core.solution import IntegerSolution
from jmetal.core.solution import Solution
from jmetal.util.termination_criterion import StoppingByEvaluations
from jmetal.util.solution import print_function_values_to_file, print_variables_to_file

In [2]:
def dcg(g):
    res = copy.deepcopy(g)
    for i in range(len(g)):
        if i > 0:
            res[i] = res[i - 1] + g[i] / math.log2(i + 1)
    return res


def compute_avg_ndcg(relevance_list: List[int]) -> float:
    """ Computes average nDCG for the fitness function. """
    l = len(relevance_list)
    sorted_scores = sorted(relevance_list, reverse=True)

    dcg_vals = dcg(relevance_list)
    idcg_vals = dcg(sorted_scores)

    dwp_vals = [dcg_vals[i] / idcg_vals[i] if idcg_vals[i] != 0 else 0 for i in range(l)]
    return sum(dwp_vals) / l

In [3]:
class RelevanceProfileProblem(IntegerProblem, ABC):
    def __init__(self, list_length: int, min_relevance: int, max_relevance: int, target_fitness: float, R: int):
        super().__init__()
        self.length = list_length
        self.lower_bound = [min_relevance] * self.length
        self.upper_bound = [max_relevance] * self.length
        self.number_of_variables = self.length
        self.number_of_objectives = 1
        self.number_of_constraints = 1
        self.target_fitness = target_fitness
        self.R = R
        
    def name(self) -> str:
        return "Relevance Profile Problem, inherits Integer Problem"
        
    def number_of_constraints(self) -> int:
        return self.number_of_constraints
    
    def number_of_objectives(self) -> int:
        return self.number_of_objectives

    def create_solution(self) -> IntegerSolution:
        values = [0] * self.length
        relevant_count = 0
        for idx in range(self.length):
            g = np.random.geometric(0.7) - 1
            
            rel = max(self.lower_bound[0], g)
            rel = min(rel, self.upper_bound[0])
            
            if rel > 0:
                relevant_count += 1
            else:
                rel = 0
            
            values[idx] = rel
                
        solution = IntegerSolution(
            lower_bound=self.lower_bound,
            upper_bound=self.upper_bound,
            number_of_objectives=self.number_of_objectives,
            number_of_constraints=self.number_of_constraints
        )
        solution.variables = values
        return solution
    
    def evaluate(self, solution: IntegerSolution) -> None:
        """ Evaluate the solution based on average nDCG with respect to target fitness. """
        relevant_count = sum(1 for x in solution.variables if x > 0)
        
        if relevant_count > self.R:
            solution.objectives[0] = float('inf')
            return
        
        andcg = compute_avg_ndcg(solution.variables)
        solution.objectives[0] = abs(self.target_fitness - andcg)


In [4]:
class CustomCrossover(Crossover, ABC):
    def __init__(self, L: int):
        super().__init__(probability=1.0)  # Always apply
        self.L = L
        
    def get_name(self) -> str:
        return "Custom Crossover for ranking profiles. "
    
    def get_number_of_parents(self) -> int:
        return 2
    
    def get_number_of_children(self) -> int:
        return 2

    def execute(self, parents: List[Solution] )-> List[Solution]:
        """ Executes the crossover, by either adding or multiplying items from two parent solutions. """
        parent1, parent2 = parents[0].variables, parents[1].variables
        child1, child2 = [], []
        for a, b in zip(parent1, parent2):
            if random.random() < 0.5:
                c1 = (a + b) % (self.L + 1)
                c2 = (a * b) % (self.L + 1)
            else:
                c1 = (a * b) % (self.L + 1)
                c2 = (a + b) % (self.L + 1)
            child1.append(c1)
            child2.append(c2)

        offspring1 = copy.deepcopy(parents[0])
        offspring2 = copy.deepcopy(parents[1])
        offspring1.variables = child1
        offspring2.variables = child2
        return [offspring1, offspring2]

class CustomMutation(Mutation, ABC):
    def __init__(self, probability, L: int):
        super().__init__(probability)
        self.L = L
        
    def get_name(self) -> str:
        return "Custom Mutation for ranking profiles. "

    def execute(self, solution: Solution) -> Solution:
        """ Executes the mutation, by randomly swapping two items in a solution or adding a random quantity to an item. """
        for i in range(len(solution.variables)):
            if random.random() < self.probability:
                if random.random() < 0.5:
                    # Swap the selected index with another random index
                    j = random.randint(0, len(solution.variables) - 1)
                    solution.variables[i], solution.variables[j] = solution.variables[j], solution.variables[i]
                else:
                    # Add a random quantity mod L + 1 to the selected index
                    solution.variables[i] = (solution.variables[i] + random.randint(0, self.L)) % (self.L + 1)
        return solution


In [6]:
min_relevance = 0
max_relevance = 3
problem = RelevanceProfileProblem(list_length= 100, min_relevance=min_relevance, max_relevance=max_relevance, target_fitness=0.8, R=50)
L = max_relevance - min_relevance

algorithm = NSGAII(
    problem=problem,
    population_size=10,
    offspring_population_size=10,
    mutation=CustomMutation(probability=1.0, L=L),
    crossover=CustomCrossover(L=L),
    termination_criterion=StoppingByEvaluations(max_evaluations=1000)
)

algorithm.run()
result = algorithm.result()

print_function_values_to_file(result, 'FUN')
print_variables_to_file(result, 'VAR')


[2025-05-07 11:48:23,463] [jmetal.core.algorithm] [DEBUG] Creating initial set of solutions...
[2025-05-07 11:48:23,465] [jmetal.core.algorithm] [DEBUG] Evaluating solutions...
[2025-05-07 11:48:23,472] [jmetal.core.algorithm] [DEBUG] Initializing progress...
[2025-05-07 11:48:23,474] [jmetal.core.algorithm] [DEBUG] Running main loop until termination criteria is met
[2025-05-07 11:48:23,884] [jmetal.core.algorithm] [DEBUG] Finished!
[2025-05-07 11:48:23,884] [jmetal.util.solution] [INFO] Output file (function values): FUN
[2025-05-07 11:48:23,884] [jmetal.util.solution] [INFO] Output file (variables): VAR
